# Blinkit Sales Data Pipeline 
This notebook demonstrates the end-to-end data processing pipeline for the Blinkit Sales Analysis project.

## Workflow Overview
1. Load raw data
2. Clean and preprocess data
3. Transform and engineer features
4. Load processed data into MySQL using PyMySQL

The final dataset is used for visualization in Power BI.


## Importing Libraries 

In [3]:
import pandas as pd
import numpy as np
import pymysql


## 1. Data Loading

The raw dataset is loaded from a CSV file for initial inspection and processing.


In [4]:
df = pd.read_excel("BlinkIT Grocery Data.xlsx")
df.head()

,Item Fat Content,Item Identifier,Item Type,Outlet Establishment Year,Outlet Identifier,Outlet Location Type,Outlet Size,Outlet Type,Item Visibility,Item Weight,Sales,Rating
0,Regular,FDX32,Fruits and Vegetables,2012,OUT049,Tier 1,Medium,Supermarket Type1,0.100014,15.10,145.4786,5.0
1,Low Fat,NCB42,Health and Hygiene,2022,OUT018,Tier 3,Medium,Supermarket Type2,0.008596,11.80,115.3492,5.0
2,Regular,FDR28,Frozen Foods,2016,OUT046,Tier 1,Small,Supermarket Type1,0.025896,13.85,165.0210,5.0
3,Regular,FDL50,Canned,2014,OUT013,Tier 3,High,Supermarket Type1,0.042278,12.15,126.5046,5.0
4,Low Fat,DRI25,Soft Drinks,2015,OUT045,Tier 2,Small,Supermarket Type1,0.033970,19.60,55.1614,5.0


## 2. Initial Data Exploration

Basic checks to understand data shape, column names, and missing values.


In [6]:

df.shape
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item Fat Content           8523 non-null   str    
 1   Item Identifier            8523 non-null   str    
 2   Item Type                  8523 non-null   str    
 3   Outlet Establishment Year  8523 non-null   int64  
 4   Outlet Identifier          8523 non-null   str    
 5   Outlet Location Type       8523 non-null   str    
 6   Outlet Size                8523 non-null   str    
 7   Outlet Type                8523 non-null   str    
 8   Item Visibility            8523 non-null   float64
 9   Item Weight                7060 non-null   float64
 10  Sales                      8523 non-null   float64
 11  Rating                     8523 non-null   float64
dtypes: float64(4), int64(1), str(7)
memory usage: 799.2 KB


In [8]:
df.isnull().sum()

Item Fat Content                0
Item Identifier                 0
Item Type                       0
Outlet Establishment Year       0
Outlet Identifier               0
Outlet Location Type            0
Outlet Size                     0
Outlet Type                     0
Item Visibility                 0
Item Weight                  1463
Sales                           0
Rating                          0
dtype: int64

## 3. Data Cleaning

In this step:
- Handled missing values
- Standardized categorical columns
- Corrected inconsistent labels


In [11]:
df.columns = (
    df.columns.str.strip().str.lower().str.replace(" ","_")
)
    

In [12]:
df.columns

Index(['item_fat_content', 'item_identifier', 'item_type',
       'outlet_establishment_year', 'outlet_identifier',
       'outlet_location_type', 'outlet_size', 'outlet_type', 'item_visibility',
       'item_weight', 'sales', 'rating'],
      dtype='str')

### Replacing Empty Cell(White Space) int Nan

In [13]:
df.replace(r'^\s*$',np.nan,regex=True,inplace=True)

,item_fat_content,item_identifier,item_type,outlet_establishment_year,outlet_identifier,outlet_location_type,outlet_size,outlet_type,item_visibility,item_weight,sales,rating
0,Regular,FDX32,Fruits and Vegetables,2012,OUT049,Tier 1,Medium,Supermarket Type1,0.100014,15.10,145.4786,5.0
1,Low Fat,NCB42,Health and Hygiene,2022,OUT018,Tier 3,Medium,Supermarket Type2,0.008596,11.80,115.3492,5.0
2,Regular,FDR28,Frozen Foods,2016,OUT046,Tier 1,Small,Supermarket Type1,0.025896,13.85,165.0210,5.0
3,Regular,FDL50,Canned,2014,OUT013,Tier 3,High,Supermarket Type1,0.042278,12.15,126.5046,5.0
4,Low Fat,DRI25,Soft Drinks,2015,OUT045,Tier 2,Small,Supermarket Type1,0.033970,19.60,55.1614,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,low fat,NCT53,Health and Hygiene,2018,OUT027,Tier 3,Medium,Supermarket Type3,0.000000,NaN,164.5526,4.0
8519,low fat,FDN09,Snack Foods,2018,OUT027,Tier 3,Medium,Supermarket Type3,0.034706,NaN,241.6828,4.0
8520,low fat,DRE13,Soft Drinks,2018,OUT027,Tier 3,Medium,Supermarket Type3,0.027571,NaN,86.6198,4.0
8521,reg,FDT50,Dairy,2018,OUT027,Tier 3,Medium,Supermarket Type3,0.107715,NaN,97.8752,4.0


In [15]:
cat_cols = [
    'item_type',
    'item_fat_content',
    'outlet_type',
    'outlet_size',
    'outlet_location_type'
]
for col in cat_cols:
    df[col] = df[col].str.lower().str.strip()

In [16]:
df['item_fat_content'].unique()

<StringArray>
['regular', 'low fat', 'lf', 'reg']
Length: 4, dtype: str

In [17]:
df['item_fat_content'].value_counts(dropna=False)

item_fat_content
low fat    5201
regular    2889
lf          316
reg         117
Name: count, dtype: int64

In [18]:
fat_map ={
    'lf':'low fat',
    'reg':'regular'
}
df['item_fat_content'] = df['item_fat_content'].replace(fat_map)

In [19]:
df['item_fat_content'].value_counts(dropna=False)

item_fat_content
low fat    5517
regular    3006
Name: count, dtype: int64

In [20]:
df['outlet_size'].unique()

<StringArray>
['medium', 'small', 'high']
Length: 3, dtype: str

In [21]:
df['outlet_type'].unique()

<StringArray>
['supermarket type1', 'supermarket type2', 'grocery store',
 'supermarket type3']
Length: 4, dtype: str

In [22]:
df[['item_weight']].isnull().sum()

item_weight    1463
dtype: int64

In [23]:
df['item_weight']=df['item_weight'].fillna(df['item_weight'].median(),inplace=True)

C:\Users\lethesh\AppData\Local\Temp\ipykernel_35120\3166640071.py:1: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  df['item_weight']=df['item_weight'].fillna(df['item_weight'].median(),inplace=True)


In [24]:
df[['item_weight']].isnull().sum()

item_weight    0
dtype: int64

In [25]:
df['outlet_establishment_year'] =df['outlet_establishment_year'].astype(int)
df['rating'] =df['rating'].astype(int)


In [26]:
df['item_fat_content'].value_counts()



item_fat_content
low fat    5517
regular    3006
Name: count, dtype: int64

In [27]:
df['outlet_size'].value_counts()

outlet_size
medium    3631
small     3139
high      1753
Name: count, dtype: int64

## 4. Data Transformation & Feature Engineering

Prepared the dataset for analytical use by:
- Creating derived columns
- Ensuring correct data types
- Selecting relevant features


In [28]:
item_df = df[[                
    'item_identifier',                 
    'item_type',
    'item_fat_content',
    'item_visibility',                
    'item_weight'
]].drop_duplicates()
    

In [29]:
outlets_df = df[[                
    'outlet_establishment_year',       
    'outlet_identifier',               
    'outlet_location_type',            
    'outlet_size',                    
    'outlet_type'                     
]].drop_duplicates()

In [30]:
sales_df = df[[                
    'item_identifier',       
    'outlet_identifier',               
    'sales',            
    'rating'                                        
]]
sales_df.insert(0,'sales_id',range(1,len(sales_df)+1))

In [31]:
item_df.head()
item_df.shape
item_df.info()

<class 'pandas.DataFrame'>
Index: 8470 entries, 0 to 8522
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   item_identifier   8470 non-null   str    
 1   item_type         8470 non-null   str    
 2   item_fat_content  8470 non-null   str    
 3   item_visibility   8470 non-null   float64
 4   item_weight       8470 non-null   float64
dtypes: float64(2), str(3)
memory usage: 397.0 KB


In [32]:
outlets_df.head()
outlets_df.shape
outlets_df.info()

<class 'pandas.DataFrame'>
Index: 16 entries, 0 to 304
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   outlet_establishment_year  16 non-null     int64
 1   outlet_identifier          16 non-null     str  
 2   outlet_location_type       16 non-null     str  
 3   outlet_size                16 non-null     str  
 4   outlet_type                16 non-null     str  
dtypes: int64(1), str(4)
memory usage: 768.0 bytes


In [33]:
item_df['item_identifier'].duplicated().sum()

np.int64(6911)

In [34]:
item_df[item_df['item_identifier'].duplicated(keep=False)].sort_values('item_identifier')

,item_identifier,item_type,item_fat_content,item_visibility,item_weight
7882,DRA12,soft drinks,low fat,0.040912,11.60
5455,DRA12,soft drinks,low fat,0.000000,11.60
4690,DRA12,soft drinks,low fat,0.068535,11.60
5860,DRA12,soft drinks,low fat,0.041178,11.60
4344,DRA12,soft drinks,low fat,0.041113,11.60
...,...,...,...,...,...
5483,NCZ54,household,low fat,0.082956,12.60
3948,NCZ54,household,low fat,0.083528,14.65
8088,NCZ54,household,low fat,0.083699,14.65
6274,NCZ54,household,low fat,0.083489,14.65


In [35]:
item_df[item_df['item_identifier'].duplicated(keep=False)].sort_values('item_identifier')

,item_identifier,item_type,item_fat_content,item_visibility,item_weight
7882,DRA12,soft drinks,low fat,0.040912,11.60
5455,DRA12,soft drinks,low fat,0.000000,11.60
4690,DRA12,soft drinks,low fat,0.068535,11.60
5860,DRA12,soft drinks,low fat,0.041178,11.60
4344,DRA12,soft drinks,low fat,0.041113,11.60
...,...,...,...,...,...
5483,NCZ54,household,low fat,0.082956,12.60
3948,NCZ54,household,low fat,0.083528,14.65
8088,NCZ54,household,low fat,0.083699,14.65
6274,NCZ54,household,low fat,0.083489,14.65


In [36]:
item_df = item_df.drop_duplicates(subset=['item_identifier'])

In [37]:
item_df['item_identifier'].duplicated().sum()

np.int64(0)

In [38]:
outlets_df['outlet_identifier'].duplicated().sum()

np.int64(6)

In [39]:
outlets_df = outlets_df.drop_duplicates(subset=['outlet_identifier'])

In [40]:
sales_df['sales_id'].duplicated().sum()

np.int64(0)

In [41]:
item_df.head()
item_df.shape
item_df.info()

<class 'pandas.DataFrame'>
Index: 1559 entries, 0 to 8452
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   item_identifier   1559 non-null   str    
 1   item_type         1559 non-null   str    
 2   item_fat_content  1559 non-null   str    
 3   item_visibility   1559 non-null   float64
 4   item_weight       1559 non-null   float64
dtypes: float64(2), str(3)
memory usage: 73.1 KB


In [42]:
outlets_df.head()


,outlet_establishment_year,outlet_identifier,outlet_location_type,outlet_size,outlet_type
0,2012,OUT049,tier 1,medium,supermarket type1
1,2022,OUT018,tier 3,medium,supermarket type2
2,2016,OUT046,tier 1,small,supermarket type1
3,2014,OUT013,tier 3,high,supermarket type1
4,2015,OUT045,tier 2,small,supermarket type1


In [43]:
sales_df.head()
sales_df.shape
sales_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sales_id           8523 non-null   int64  
 1   item_identifier    8523 non-null   str    
 2   outlet_identifier  8523 non-null   str    
 3   sales              8523 non-null   float64
 4   rating             8523 non-null   int64  
dtypes: float64(1), int64(2), str(2)
memory usage: 333.1 KB


In [44]:
outlets_df.info()

<class 'pandas.DataFrame'>
Index: 10 entries, 0 to 91
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   outlet_establishment_year  10 non-null     int64
 1   outlet_identifier          10 non-null     str  
 2   outlet_location_type       10 non-null     str  
 3   outlet_size                10 non-null     str  
 4   outlet_type                10 non-null     str  
dtypes: int64(1), str(4)
memory usage: 480.0 bytes


## 6. Database Connection (MySQL)

Establising a connection to MySQL using PyMySQL for data storage.
## 7. Data Loading into MySQL

Inserting cleaned and transformed data into MySQL using batch inserts.



In [46]:
class DatabaseTable:
    def __init__(self, table_name, columns):
        self.table = table_name
        self.columns = columns
        
    def connect(self, autocommit=True):
        return pymysql.connect(
            host="localhost",
            user="root",
            password="YOUR_PASSWORD",
            database="YOUR_DB_NAME",
            autocommit=autocommit
        )
        
    def insert_dataframe(self, df, batch_size=1000):
        conn = None
        try:
            conn = self.connect(autocommit=False)
            cur = conn.cursor()
            cols = ", ".join(self.columns)
            placeholders = ", ".join(["%s"] * len(self.columns))
            sql = f"INSERT INTO {self.table} ({cols}) VALUES ({placeholders})"
            data = df[self.columns].values.tolist()
            for i in range(0, len(data), batch_size):
                cur.executemany(sql, data[i:i + batch_size])
            conn.commit()
            print(f" {len(df)} rows inserted into {self.table}")
        except Exception as e:
            if conn:
                conn.rollback()
            print(f"Bulk insert failed for {self.table}:", e)
        finally:
            if conn:
                conn.close()
                
    def insert(self):
        try:
            values = [input(f"Enter {col}: ") for col in self.columns]
            cols = ", ".join(self.columns)
            vals = ", ".join([f"'{v}'" for v in values])
            sql = f"INSERT INTO {self.table} ({cols}) VALUES ({vals})"
            conn = self.connect()
            cur = conn.cursor()
            cur.execute(sql)
            conn.close()
            print("Insert successful")
        except Exception as e:
            print("Insert failed:", e)
            
    def select_all(self):
        try:
            conn = self.connect()
            cur = conn.cursor()
            cur.execute(f"SELECT * FROM {self.table}")
            rows = cur.fetchall()
            conn.close()
            if not rows:
                print("No records found")
            else:
                for row in rows:
                    print(row)
        except Exception as e:
            print("Select failed:", e)
            
    def update(self):
        try:
            where_col = input("Where column: ")
            where_val = input("Where value: ")
            column = input("Column to update: ")
            new_value = input("New value: ")
            sql = f"""
            UPDATE {self.table}
            SET {column} = '{new_value}'
            WHERE {where_col} = '{where_val}'
            """
            conn = self.connect()
            cur = conn.cursor()
            cur.execute(sql)
            conn.close()
            print("Update successful")
        except Exception as e:
            print("Update failed:", e)
            
    def delete(self):
        try:
            where_col = input("Where column: ")
            where_val = input("Where value: ")
            sql = f"DELETE FROM {self.table} WHERE {where_col} = '{where_val}'"
            conn = self.connect()
            cur = conn.cursor()
            cur.execute(sql)
            conn.close()
            print("Delete successful")
        except Exception as e:
            print("Delete failed:", e)

In [47]:
tables = {
    "items": DatabaseTable(
        "items",
        ["item_identifier", "item_type", "item_fat_content", "item_visibility", "item_weight"]
    ),
    "outlets": DatabaseTable(
        "outlets",
        ["outlet_identifier", "outlet_establishment_year",
         "outlet_location_type", "outlet_size", "outlet_type"]
    ),
    "sales": DatabaseTable(
        "sales",
        ["sales_id", "item_identifier", "outlet_identifier", "sales", "rating"]
    )
}


In [10]:
while True:
    print("""
1. Insert (manual)
2. Select
3. Update
4. Delete
5. Bulk insert from DataFrame
0. Exit
""")

    ch = input("Choose operation: ").strip()

    if ch == "0":
        print("Exiting...")
        break

    if ch in {"1", "2", "3", "4"}:
        table_name = input("Table (items / outlets / sales): ").strip()

        if table_name not in tables:
            print("Invalid table")
            continue

        table = tables[table_name]

        if ch == "1":
            table.insert()
        elif ch == "2":
            table.select_all()
        elif ch == "3":
            table.update()
        elif ch == "4":
            table.delete()

    elif ch == "5":
        print("Bulk inserting DataFrames...")

        try:
            tables["items"].insert_dataframe(item_df)
            tables["outlets"].insert_dataframe(outlets_df)
            tables["sales"].insert_dataframe(sales_df)
            print("Bulk insert completed")

        except Exception as e:
            print("Bulk insert failed:", e)

    else:
        print("Invalid choice")



1. Insert (manual)
2. Select
3. Update
4. Delete
5. Bulk insert from DataFrame
0. Exit



Choose operation:  0


Exiting...


## Pipeline Completed

The cleaned and transformed Blinkit sales data has been successfully loaded into MySQL.

This data is further used in Power BI for visualization and business analysis.
